In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Loading MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Split training/validation set
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=10000, random_state=42
)


In [ ]:
# Normalization
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

# add channel dims (28, 28, 1)
train_images = np.expand_dims(train_images, -1)
val_images = np.expand_dims(val_images, -1)
test_images = np.expand_dims(test_images, -1)

# one-hot encode cuz only recongnize 0~9
train_labels = tf.one_hot(train_labels, 10)
val_labels = tf.one_hot(val_labels, 10)
test_labels = tf.one_hot(test_labels, 10)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


In [ ]:
#Compile your model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Train the model and add validation data
history = model.fit(train_images, train_labels,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_images, val_labels))


In [ ]:
model.save('mnist_cnn_model.keras')

# Reload model
model = tf.keras.models.load_model('mnist_cnn_model.keras')


In [ ]:
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(test_labels.numpy(), axis=1)

# Test predict result
for i in range(10):
    print(f"Actual: {actual_labels[i]}, Predicted: {predicted_labels[i]}")


In [ ]:
#Show the images of the test result
for i in range(10):
    plt.imshow(test_images[i].squeeze(), cmap='gray')
    plt.title(f"Actual: {actual_labels[i]}, Predicted: {predicted_labels[i]}")
    plt.axis('off')
    plt.show()


In [ ]:
#Create a writting board to write and recognize
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('mnist_cnn_model.keras')

class DigitRecognizerApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Handwritten recognition")
        self.canvas_width = 280
        self.canvas_height = 280

        self.canvas = tk.Canvas(master, width=self.canvas_width, height=self.canvas_height, bg='white')
        self.canvas.pack()

        self.image = Image.new("L", (self.canvas_width, self.canvas_height), 255)
        self.draw = ImageDraw.Draw(self.image)

        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset)

        self.result_label = tk.Label(master, text="Predict Result：", font=("Arial", 16))
        self.result_label.pack()

        self.predict_button = tk.Button(master, text="Recognize", command=self.predict_digit)
        self.predict_button.pack(side=tk.LEFT, padx=10, pady=10)

        self.clear_button = tk.Button(master, text="Clear", command=self.clear_canvas)
        self.clear_button.pack(side=tk.RIGHT, padx=10, pady=10)

        self.last_x, self.last_y = None, None

    def paint(self, event):
        if self.last_x and self.last_y:
            self.canvas.create_line(self.last_x, self.last_y, event.x, event.y,
                                    width=12, fill='black', capstyle=tk.ROUND, smooth=tk.TRUE, splinesteps=36)
            self.draw.line([self.last_x, self.last_y, event.x, event.y], fill=0, width=12)
        self.last_x = event.x
        self.last_y = event.y

    def reset(self, event):
        self.last_x, self.last_y = None, None

    def clear_canvas(self):
        self.canvas.delete("all")
        self.draw.rectangle([0, 0, self.canvas_width, self.canvas_height], fill=255)
        self.result_label.config(text="Predict Result：")

    def predict_digit(self):
        # Pre-process：rescale、reverse 、normalization
        image = self.image.resize((28, 28))
        image = ImageOps.invert(image)
        img_array = np.array(image) / 255.0
        img_array = img_array.reshape(1, 28, 28, 1)

        # Prediction
        prediction = model.predict(img_array)
        digit = np.argmax(prediction)
        confidence = np.max(prediction)

        self.result_label.config(text=f"Predict Result：{digit}（Confidence Value：{confidence:.2f}）")

if __name__ == '__main__':
    root = tk.Tk()
    app = DigitRecognizerApp(root)
    root.mainloop()
